<a href="https://colab.research.google.com/github/RedzaAuditore/Task-Machine-Learning/blob/main/Week%207/%20Distributed_XGBoost_with_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ray

     |████████████████████████████████| 54.7 MB 35 kB/s 
     |████████████████████████████████| 72 kB 621 kB/s 


In [ ]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 25.1 MB/s 


In [ ]:
pip install sklearn

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-10 14:37:29,594	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-10 14:37:31,638	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=287) [14:37:31] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=287) [14:37:31] Tree method is automatically selected to be 'approx' for distributed training.
(pid=311) [14:37:31] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [ ]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-10 14:37:33,103	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-10 14:37:35,404	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-10 14:37:35,510	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-10 14:37:35,530	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-10 14:37:35,532	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_bee83_00000,PENDING,,0.0070233,7,0.907962
train_model_bee83_00001,PENDING,,0.00202672,5,0.961565
train_model_bee83_00002,PENDING,,0.000248038,2,0.524566
train_model_bee83_00003,PENDING,,0.01038,8,0.888123


(pid=391) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=391)   warnings.warn(LEGACY_WARNING)
(pid=391) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=391)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=391) 2021-11-10 14:37:37,847	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=391) 2021-11-10 14:37:40,360	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_bee83_00000,RUNNING,172.28.0.2:391,0.0070233,7,0.907962
train_model_bee83_00001,PENDING,,0.00202672,5,0.961565
train_model_bee83_00002,PENDING,,0.000248038,2,0.524566
train_model_bee83_00003,PENDING,,0.01038,8,0.888123


Result for train_model_bee83_00000:
  date: 2021-11-10_14-37-41
  done: false
  experiment_id: 56931ad7f7f44380aa0dc2fa7de6f168
  hostname: c5c57dd5df1b
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 391
  time_since_restore: 3.597384452819824
  time_this_iter_s: 3.597384452819824
  time_total_s: 3.597384452819824
  timestamp: 1636555061
  timesteps_since_restore: 0
  train-error: 0.024605
  train-logloss: 0.686876
  training_iteration: 1
  trial_id: bee83_00000
  
Result for train_model_bee83_00000:
  date: 2021-11-10_14-37-41
  done: true
  experiment_id: 56931ad7f7f44380aa0dc2fa7de6f168
  experiment_tag: 0_eta=0.0070233,max_depth=7,subsample=0.90796
  hostname: c5c57dd5df1b
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 391
  time_since_restore: 3.6553170680999756
  time_this_iter_s: 0.004830360412597656
  time_total_s: 3.6553170680999756
  timestamp: 1636555061
  timesteps_since_restore: 0
  train-error: 0.008787
  train-logloss: 0.63437
  training_iteratio

...

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_bee83_00003,RUNNING,172.28.0.2:705,0.01038,8,0.888123,,,,
train_model_bee83_00000,TERMINATED,172.28.0.2:391,0.0070233,7,0.907962,10,3.65532,0.63437,0.008787
train_model_bee83_00001,TERMINATED,172.28.0.2:497,0.00202672,5,0.961565,10,3.66381,0.675458,0.010545
train_model_bee83_00002,TERMINATED,172.28.0.2:601,0.000248038,2,0.524566,10,3.69773,0.691156,0.043937


Result for train_model_bee83_00003:
  date: 2021-11-10_14-37-58
  done: false
  experiment_id: 15fc5eb02055450aaa4d59c1e2b4289a
  hostname: c5c57dd5df1b
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 705
  time_since_restore: 3.595700740814209
  time_this_iter_s: 3.595700740814209
  time_total_s: 3.595700740814209
  timestamp: 1636555078
  timesteps_since_restore: 0
  train-error: 0.026362
  train-logloss: 0.683962
  training_iteration: 1
  trial_id: bee83_00003
  
Result for train_model_bee83_00003:
  date: 2021-11-10_14-37-58
  done: true
  experiment_id: 15fc5eb02055450aaa4d59c1e2b4289a
  experiment_tag: 3_eta=0.01038,max_depth=8,subsample=0.88812
  hostname: c5c57dd5df1b
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 705
  time_since_restore: 3.654897928237915
  time_this_iter_s: 0.005064725875854492
  time_total_s: 3.654897928237915
  timestamp: 1636555078
  timesteps_since_restore: 0
  train-error: 0.008787
  train-logloss: 0.609248
  training_iteration: 

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_bee83_00000,TERMINATED,172.28.0.2:391,0.0070233,7,0.907962,10,3.65532,0.63437,0.008787
train_model_bee83_00001,TERMINATED,172.28.0.2:497,0.00202672,5,0.961565,10,3.66381,0.675458,0.010545
train_model_bee83_00002,TERMINATED,172.28.0.2:601,0.000248038,2,0.524566,10,3.69773,0.691156,0.043937
train_model_bee83_00003,TERMINATED,172.28.0.2:705,0.01038,8,0.888123,10,3.6549,0.609248,0.008787


(pid=705) 2021-11-10 14:37:58,108	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.67 seconds (1.10 pure XGBoost training time).
2021-11-10 14:37:58,285	INFO tune.py:630 -- Total run time: 22.78 seconds (22.59 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.007023299237114316, 'subsample': 0.907962339015004, 'max_depth': 7}
